# Convert a 3D .nrrd file to a 2D tiff stack 
*Last edited 17 Nov 2024 by K. Wolcott*    
Convert a 3D .nrrd file made in 3D Slicer into a stack of 2D tiff images. This script is an alternate to manually converting .nrrd to tiff stack in Fiji.

In [ ]:
# Installs
!pip install pillow
!pip install numpy
!pip install pynrrd
!pip install os

In [ ]:
# Imports
import numpy as np
from PIL import Image
import nrrd
import os

# Convert .nrrd file from 3D Slicer to a tiff stack (Alternative to using Fiji GUI)
def convert_nrrd(fn, inpath = "./", outpath = "./", nbits = "16"):
    """
    This function converts an .nrrd file (from a micro-CT scan) to a tiff stack for upload to Morphosource.
    Define your inpath (folder where your .nrrd file is saved), fin (your .nrrd filename), 
    outpath (folder where you want to save your tiff stack), and nbits (how many bits your dataset has; currently only tested for unsigned 16-bit)
    Example input variables:
    fn = 'thecac_ftg_rf_220622_2yPTAtest_fullres_cropped_masked.nrrd'
    inpath = '/media/lab-admin/data/THECAC/thecac_ftg_rf_220622_2yPTAtest/slicer'
    outpath = '/media/lab-admin/data/THECAC/thecac_ftg_rf_220622_2yPTAtest/slicer/ms/tiffs_py'
    nbits = '16'
    """
    base_fn = os.path.splitext(fn)[0]
    fpath = inpath + "/" + fn
    data, header = nrrd.read(fpath)
    print("Data shape for {}: \n{}".format(fn, data.shape))
    # Confirm that 16-bit images are being used
    if nbits != "16":
        print("You selected that you aren't using 16-bit unsigned images, please check PIL documentation and change image mode accordingly. https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes")
    if not os.path.exists(outpath):
        os.makedirs(outpath)
        print("Making folder to save tiff stack in: ", outpath)
    # Split .nrrd into individual tiffs and save to folder
    for i in range(data.shape[2]):
        img = np.array(data[:,:,i])
        # TO DO: Adjust image mode here if not using 16-bit unsigned images
        img = Image.fromarray(np.uint16(img), mode = "I;16") # tested for 16bit unisgned from Versa Xradia 620
        fpath_out = outpath + "/" + base_fn + "_" + str(i)+ ".tiff"
        img.save(fpath_out)
    print("Tiff stack of {} images saved to: {}".format(data.shape[2], outpath))

In [ ]:
# Convert .nrrd file from 3D slicer to a tiff stack
# TO DO: Change values for below variables to match your .nrrd file input and tiff stack output locations
fn = 'thecac_ftg_rf_220622_2yPTAtest_fullres_cropped_masked.nrrd'
inpath = '/media/lab-admin/data/THECAC/thecac_ftg_rf_220622_2yPTAtest/slicer'
outpath = '/media/lab-admin/data/THECAC/thecac_ftg_rf_220622_2yPTAtest/slicer/ms/tiffs_py'
nbits = '16'

# Convert the .nrrd file
convert_nrrd(fn, inpath, outpath)

In [ ]:
# Zip the tiff stack so it's ready for MorphoSource upload
%cd $outpath
print("Total folder size of tiff stack in: ", outpath)
!du -sh -- .

print("Zipping folder contents...")
%cd ../
base_fn = os.path.splitext(fn)[0]
zipped_fn = base_fn + '_zipped_stack.zip'
unzipped_fn = os.path.basename(outpath)
!zip -r $zipped_fn $unzipped_fn

print("Total folder size after zipping: ")
!du -h $zipped_fn